In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from sklearn.model_selection import train_test_split

In [2]:
df_train = pd.read_csv('train.csv', sep='\t')#, index_col='id')

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
id             200000 non-null int64
name           200000 non-null object
description    200000 non-null object
target         200000 non-null int64
dtypes: int64(2), object(2)
memory usage: 6.1+ MB


In [4]:
from tqdm import tqdm

def ClearTag(TextData): #очиста от тэгов в столбце
    texts_without_tags = []
    
    for t in tqdm(TextData):
        texts_without_tags.append(BeautifulSoup(t, 'lxml').text)
    
    return texts_without_tags

In [5]:
df_train['name'] = ClearTag(df_train['name'])

100%|██████████| 200000/200000 [01:20<00:00, 2495.44it/s]


In [7]:
import pymorphy2 # Морфологический анализатор.
from collections import Counter # Не считать же частоты самим.
#import math # Корень квадратный.

morph=pymorphy2.MorphAnalyzer()

In [8]:
def getMeaningfullWords(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁёa-zA-Z]+\-[А-Яа-яЁёa-zA-Z]+|[А-Яа-яЁёa-zA-Z]+', text)
    for t in tokens:
        pv=morph.parse(t)
        for p in pv:
            if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                words.append(p.normal_form)
                break
                
    tokens_eng=re.findall('[a-zA-Z]+\-[a-zA-Z]+|[a-zA-Z]+', text)
    for t in tokens_eng:
        words.append(t)
    
    return ' '.join(words)

In [9]:
getMeaningfullWords("Пиздец Account подкрался manger не заметно. Красивая ты вошла в мою грешную жизнь")

'пиздец подкрасться красивый войти век мой грешный жизнь Account manger'

In [10]:
num = df_train['name'].size #считываем размер выборки
#num = 10

clean_train_words = [] #собираем "мешок слов"

for i in tqdm(range(0, num)):
    clean_train_words.append(getMeaningfullWords(df_train['name'][i]))

100%|██████████| 200000/200000 [05:56<00:00, 561.28it/s]


In [11]:
#сохранение резельтатов вычислений
# !!!ЗАПУСКАТЬ ОТДЕЛЬНО!!!
# ПРИ НЕОБХОДИМОСТИ СОХРАНИТЬ ДАННЫЕ В ФАЙЛ

filename = 'clean_train_words.csv'
file_save = open(filename, "w")

for item in clean_train_words:
    file_save.write(item+'\n')

file_save.close()

In [6]:
#LOAD clean_train_words from the file
filename = 'clean_train_words.csv'
file_load = open(filename)

text = file_load.read()
clean_train_words = text.split('\n')[1:]

file_load.close()

In [ ]:
#SAVE train_data_features into the file
import _pickle as cPickle

filename = 'train_data_features.csv'

with open('train_data_features.csv', 'wb') as f:
    cPickle.dump(train_data_features, f)

In [ ]:
#LOAD train_data_features from the file
import _pickle as cPickle

filename = 'train_data_features.csv'

with open(filename, 'wb') as f:
    train_data_features = cPickle.load(f)

In [ ]:
df_train['name'][12]

In [ ]:
#clean_train_words

In [12]:
from sklearn.feature_extraction.text import CountVectorizer 

In [18]:
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000,
                             ngram_range=(1,3)) 

In [19]:
%%time
train_data_features = vectorizer.fit_transform(clean_train_words)

Wall time: 6.15 s


In [ ]:
%%time
#train_data_features = train_data_features.toarray()

In [20]:
train_data_features.shape

(200000, 5000)

In [21]:
vectorizer.vocabulary_

{'отдел': 3161,
 'секция': 4172,
 'век': 769,
 'магазин': 2370,
 'век магазин': 785,
 'наладчик': 2906,
 'станок': 4399,
 'исполняющий': 1708,
 'манипулятор': 2434,
 'секунда': 4132,
 'наладчик станок': 2911,
 'станок исполняющий': 4400,
 'исполняющий манипулятор': 1721,
 'манипулятор секунда': 2435,
 'наладчик станок исполняющий': 2912,
 'станок исполняющий манипулятор': 4401,
 'исполняющий манипулятор секунда': 1722,
 'разработчик': 3837,
 'разработчик секунда': 3879,
 'фрезеровщик': 4786,
 'мерчендайзер': 2667,
 'продавец': 3515,
 'консультант': 2107,
 'продавец консультант': 3548,
 'мастер': 2466,
 'эксплуатация': 4926,
 'здание': 1538,
 'торговый': 4637,
 'представитель': 3422,
 'торговый представитель': 4650,
 'менеджер': 2512,
 'продажа': 3596,
 'менеджер продажа': 2572,
 'работа': 3795,
 'ключевой': 2038,
 'клиент': 2017,
 'менеджер работа': 2627,
 'работа секунда': 3797,
 'секунда ключевой': 4144,
 'ключевой клиент': 2039,
 'менеджер работа секунда': 2628,
 'работа секунда клю

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve

In [23]:
X_train, X_test, y_train, y_test = train_test_split(train_data_features, df_train['target'], test_size=0.33, random_state=123)

In [24]:
print('X_train ', X_train.shape)
print('X_test ',X_test.shape)
print('y_train ', y_train.shape)
print('y_test ',y_test.shape)

X_train  (134000, 5000)
X_test  (66000, 5000)
y_train  (134000,)
y_test  (66000,)


In [25]:
%%time
from sklearn.ensemble import RandomForestClassifier
model_RFC = RandomForestClassifier(n_estimators = 100, n_jobs = -1) 

Wall time: 459 ms


In [26]:
%%time
model_RFC.fit( X_train, y_train )

Wall time: 2min 56s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
predict = model_RFC.predict(X_test)

In [28]:
print(classification_report(y_test, predict))

             precision    recall  f1-score   support

          0       0.97      0.95      0.96     35162
          1       0.94      0.96      0.95     30838

avg / total       0.95      0.95      0.95     66000



In [29]:
roc_auc_score(y_test, predict)

0.9539689095054568

 <b color='red'>НЕЗАПУСКАТЬ!!!</b>

In [ ]:
df_train = pd.read_csv('train.csv', sep='\t', index_col='id')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train.drop('target', axis=1), df_train['target'], 
                                                    test_size=0.30, random_state=123)

X_test['target'] = y_test

In [ ]:
X_test.to_csv('train1900.csv', sep='\t', index_label='id')

In [ ]:
import _pickle as cPickle
with open('RFC.pcl', 'wb') as f:
    cPickle.dump(model_RFC, f)


In [ ]:
import _pickle as cPickle
with open('RFC.pcl', 'rb') as f:
    model_RFC = cPickle.load(f)